In [50]:
import glob
import pandas as pd
import numpy as np
import re
import plotly_express as px
import datetime
import warnings
import os
import json
import streamlit as st
st.set_page_config(layout="wide")

pd.options.display.float_format = "{:,.1f}".format

warnings.filterwarnings('ignore')

global teamnamedict
# C:\Users\mg\JupyterLabDir\Rest\Pr Winning\teamnamedict_streamlit.json
with open('teamnamedict_streamlit.json') as f:
    teamnamedict = json.load(f)

# def _max_width_():
#     max_width_str = f"max-width: 1300px;"
#     st.markdown(
#         f"""
#     <style>
#     .reportview-container .main .block-container{{
#         {max_width_str}
#     }}
#     </style>
#     """,
#         unsafe_allow_html=True,
#     )

# _max_width_()
global widthfig
widthfig = 700
heightfig = 500

# get all the gamestatistics from in dropdown specified league and season
# setup the database connection.  There's no need to setup cursors with pandas psql.
tables = list(glob.glob("htdatan/*"))
tables

['htdatan\\b_1415_24102021.csv',
 'htdatan\\b_1516_24102021.csv',
 'htdatan\\b_1617_24102021.csv',
 'htdatan\\b_1718_24102021.csv',
 'htdatan\\b_1819_24102021.csv',
 'htdatan\\b_1920_24102021.csv',
 'htdatan\\b_2021_24102021.csv',
 'htdatan\\b_2122.csv',
 'htdatan\\l1_1314_24102021.csv',
 'htdatan\\l1_1415_24102021.csv',
 'htdatan\\l1_1516_24102021.csv',
 'htdatan\\l1_1617_24102021.csv',
 'htdatan\\l1_1718_24102021.csv',
 'htdatan\\l1_1819_24102021.csv',
 'htdatan\\l1_1920_24102021.csv',
 'htdatan\\l1_2021_24102021.csv',
 'htdatan\\l1_2122.csv',
 'htdatan\\ll_1314_24102021.csv',
 'htdatan\\ll_1415_24102021.csv',
 'htdatan\\ll_1516_24102021.csv',
 'htdatan\\ll_1617_24102021.csv',
 'htdatan\\ll_1718_24102021.csv',
 'htdatan\\ll_1819_24102021.csv',
 'htdatan\\ll_1920_24102021.csv',
 'htdatan\\ll_2021_24102021.csv',
 'htdatan\\ll_2122.csv',
 'htdatan\\not_relevant',
 'htdatan\\pl_1314_24102021.csv',
 'htdatan\\pl_1415_24102021.csv',
 'htdatan\\pl_1516_24102021.csv',
 'htdatan\\pl_1617_2410

In [51]:
# take only the 0 part of the every list entry
global saissons
saissons = []
for x in range(0, len(tables)):
    saissons.append(tables[x].split("\\")[1].split("_24102021.csv")[0])
saissons

['b_1415',
 'b_1516',
 'b_1617',
 'b_1718',
 'b_1819',
 'b_1920',
 'b_2021',
 'b_2122.csv',
 'l1_1314',
 'l1_1415',
 'l1_1516',
 'l1_1617',
 'l1_1718',
 'l1_1819',
 'l1_1920',
 'l1_2021',
 'l1_2122.csv',
 'll_1314',
 'll_1415',
 'll_1516',
 'll_1617',
 'll_1718',
 'll_1819',
 'll_1920',
 'll_2021',
 'll_2122.csv',
 'not_relevant',
 'pl_1314',
 'pl_1415',
 'pl_1516',
 'pl_1617',
 'pl_1718',
 'pl_1819',
 'pl_1920',
 'pl_2021',
 'pl_2122.csv',
 'sa_1314',
 'sa_1415',
 'sa_1516',
 'sa_1617',
 'sa_1718',
 'sa_1819',
 'sa_1920',
 'sa_2021',
 'sa_2122.csv']

In [52]:
cleaned_names_saissons = []
for saison in saissons:
    saison = saison.replace("_", " ")
    saison = saison.replace(".csv", "")
    # saison = saison.strip()
    cleaned_names_saissons.append(saison)

cleaned_names_saissons

['b 1415',
 'b 1516',
 'b 1617',
 'b 1718',
 'b 1819',
 'b 1920',
 'b 2021',
 'b 2122',
 'l1 1314',
 'l1 1415',
 'l1 1516',
 'l1 1617',
 'l1 1718',
 'l1 1819',
 'l1 1920',
 'l1 2021',
 'l1 2122',
 'll 1314',
 'll 1415',
 'll 1516',
 'll 1617',
 'll 1718',
 'll 1819',
 'll 1920',
 'll 2021',
 'll 2122',
 'not relevant',
 'pl 1314',
 'pl 1415',
 'pl 1516',
 'pl 1617',
 'pl 1718',
 'pl 1819',
 'pl 1920',
 'pl 2021',
 'pl 2122',
 'sa 1314',
 'sa 1415',
 'sa 1516',
 'sa 1617',
 'sa 1718',
 'sa 1819',
 'sa 1920',
 'sa 2021',
 'sa 2122']

In [53]:
# map league shortcuts to real names:
shortcut_league_dict = {
    "ll": "La-Liga",
    "sa": "Serie-A",
    "pl": "Premier-League",
    "l1": "League-1",
    "b": "Bundesliga",
}

# list(map(cleaned_names_saissons, shortcut_league_dict) )
cleaned_names_saissons = [e.replace(key, val) for e in cleaned_names_saissons for key, val in shortcut_league_dict.items() if key in e]
cleaned_names_saissons

['Bundesliga 1415',
 'Bundesliga 1516',
 'Bundesliga 1617',
 'Bundesliga 1718',
 'Bundesliga 1819',
 'Bundesliga 1920',
 'Bundesliga 2021',
 'Bundesliga 2122',
 'League-1 1314',
 'League-1 1415',
 'League-1 1516',
 'League-1 1617',
 'League-1 1718',
 'League-1 1819',
 'League-1 1920',
 'League-1 2021',
 'League-1 2122',
 'La-Liga 1314',
 'La-Liga 1415',
 'La-Liga 1516',
 'La-Liga 1617',
 'La-Liga 1718',
 'La-Liga 1819',
 'La-Liga 1920',
 'La-Liga 2021',
 'La-Liga 2122',
 'Premier-League 1314',
 'Premier-League 1415',
 'Premier-League 1516',
 'Premier-League 1617',
 'Premier-League 1718',
 'Premier-League 1819',
 'Premier-League 1920',
 'Premier-League 2021',
 'Premier-League 2122',
 'Serie-A 1314',
 'Serie-A 1415',
 'Serie-A 1516',
 'Serie-A 1617',
 'Serie-A 1718',
 'Serie-A 1819',
 'Serie-A 1920',
 'Serie-A 2021',
 'Serie-A 2122']

In [54]:
def find_key(input_dict, value):
    for key, val in input_dict.items():
        if val == value: return key
    return "None"


"{}_{}".format(find_key(shortcut_league_dict, cleaned_names_saissons[0].split(" ")[0]),
cleaned_names_saissons[0].split(" ")[1]
)

'b_1415'

In [60]:
cleaned_names_saissons 

# sort list by int substring
sorted(cleaned_names_saissons, key=lambda x: int(x.split(" ")[-1]), reverse=True )

['Bundesliga 2122',
 'League-1 2122',
 'La-Liga 2122',
 'Premier-League 2122',
 'Serie-A 2122',
 'Bundesliga 2021',
 'League-1 2021',
 'La-Liga 2021',
 'Premier-League 2021',
 'Serie-A 2021',
 'Bundesliga 1920',
 'League-1 1920',
 'La-Liga 1920',
 'Premier-League 1920',
 'Serie-A 1920',
 'Bundesliga 1819',
 'League-1 1819',
 'La-Liga 1819',
 'Premier-League 1819',
 'Serie-A 1819',
 'Bundesliga 1718',
 'League-1 1718',
 'La-Liga 1718',
 'Premier-League 1718',
 'Serie-A 1718',
 'Bundesliga 1617',
 'League-1 1617',
 'La-Liga 1617',
 'Premier-League 1617',
 'Serie-A 1617',
 'Bundesliga 1516',
 'League-1 1516',
 'La-Liga 1516',
 'Premier-League 1516',
 'Serie-A 1516',
 'Bundesliga 1415',
 'League-1 1415',
 'La-Liga 1415',
 'Premier-League 1415',
 'Serie-A 1415',
 'League-1 1314',
 'La-Liga 1314',
 'Premier-League 1314',
 'Serie-A 1314']

In [56]:


print(saissons)
global saison
saison = st.sidebar.selectbox("Saison", list(saissons), 8)

print(saison)
df_complete_saison = pd.read_csv(
    "htdatan/"+saison, index_col=0, encoding='utf-8')

2021-11-14 15:52:12.088 
  command:

    streamlit run C:\Users\markus.gebele\Anaconda3\envs\btcpred\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


['b_1415', 'b_1516', 'b_1617', 'b_1718', 'b_1819', 'b_1920', 'b_2021', 'b_2122.csv', 'l1_1314', 'l1_1415', 'l1_1516', 'l1_1617', 'l1_1718', 'l1_1819', 'l1_1920', 'l1_2021', 'l1_2122.csv', 'll_1314', 'll_1415', 'll_1516', 'll_1617', 'll_1718', 'll_1819', 'll_1920', 'll_2021', 'll_2122.csv', 'not_relevant', 'pl_1314', 'pl_1415', 'pl_1516', 'pl_1617', 'pl_1718', 'pl_1819', 'pl_1920', 'pl_2021', 'pl_2122.csv', 'sa_1314', 'sa_1415', 'sa_1516', 'sa_1617', 'sa_1718', 'sa_1819', 'sa_1920', 'sa_2021', 'sa_2122.csv']
l1_1314


FileNotFoundError: [Errno 2] No such file or directory: 'htdatan/l1_1314'